#Collect API Data from Deadlock-API Project, filter, format, and insert into Deadlock.db

#Requirements import from .txt
#Fetch data based on URL (site+endpoint)
##Fetch hero data
#Filter fetched data based on filters
##Filter hero data
#prints or formatting

#Requirements import from .txt

In [1]:
!pip install -r requirements.txt

In [2]:
import requests
import pandas as pd
import numpy as np

#Fetch data based on URL (site+endpoint), outputs DF

In [6]:
#Fetches data and returns DF
def fetch_data(url):
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
       heroes_data = response.json()  # Converts the JSON response to a Python dictionary
    else:
        print(f"Failed to retrieve data: {response.status_code}")
    
    df = pd.DataFrame([heroes_data])
    return df

##Fetch Hero data

In [7]:

site = "https://assets.deadlock-api.com"
endpoint = "/v2/heroes/by-name"
hero = "/Lash"
url = site+endpoint+hero

heroes_df = fetch_data(url)



##Filter Data

In [8]:
def filter_data(filters, df):
    filtered_data = []
    for _, item in df.iterrows():
        filtered_item = {key: item[key] for key in filters if key in item}
        filtered_data.append(filtered_item)
    
    return pd.DataFrame(filtered_data)


##Format Data

In [9]:
def format_data(df):
    df.rename(columns={'id' : 'hero_id'}, inplace=True)
    return df

##Filter and format Hero Data

In [10]:

#columns to fetch
#filters = [""]

#heroes_df = filter_data(filters, heroes_df)
#heroes_df = format_data(heroes_df)

###Dispaly column headers.

In [11]:
# Display all columns
pd.set_option('display.max_columns', None)
heroes_df.head() 

,id,class_name,name,description,recommended_upgrades,player_selectable,disabled,in_development,needs_testing,assigned_players_only,limited_testing,complexity,skin,images,items,starting_stats,item_slot_info,physics,colors,shop_stat_display,stats_display,hero_stats_ui,level_info,scaling_stats,purchase_bonuses,standard_level_up_upgrades
0,31,hero_lash,Lash,"{'lore': 'Jacob Lash is an asshole.', 'role': ...","[upgrade_headshot_booster, upgrade_hollow_poin...",True,False,False,False,False,False,3,0,{'icon_hero_card': 'https://assets.deadlock-ap...,{'ability_climb_rope': 'citadel_ability_climb_...,"{'max_move_speed': {'value': 7.3, 'display_sta...","{'spirit': {'max_purchases_for_tier': [6, 6, 6...","{'collision_height': 80.0, 'collision_radius':...","{'glow_enemy': [255, 0, 0], 'glow_friendly': [...",{'spirit_stats_display': {'display_stats': ['E...,"{'health_header_stats': ['EMaxHealth', 'EBaseH...",{'weapon_stat_display': 'EMeleeDamage_DEPRECAT...,{'1': {'bonus_currencies': ['EAbilityUnlocks']...,{},{'spirit': [{'value_type': 'MODIFIER_VALUE_TEC...,{'MODIFIER_VALUE_BASE_BULLET_DAMAGE_FROM_LEVEL...


###print header

In [10]:
#print(heroes_df.head(1))

### Convert fitlered hero data to json

In [11]:
heroes_json = heroes_df.to_dict(orient='records')

### insert into database

In [14]:
import sqlite3

db = "Deadlock.db"

#create or connect to database file
conn = sqlite3.connect(db)
cursor = conn.cursor()

def export_to_db(df,db):
    
    for hero in df:
        cursor.execute("""
            INSERT INTO heroes (hero_id, name, player_selectable, disabled)
            VALUES (?, ?, ?, ?)
        """, (
                 hero['hero_id'], 
            hero['name'],
            hero['player_selectable'], 
            hero['disabled'], 
        ))
    print("Data was successfully loaded")

export_to_db(heroes_json, db)

Data was successfully loaded


In [15]:
conn.commit()